## Descriptions:
WE DON'T USE SIMPLE DATA IN THIS PROJECT -- can disregard this notebook

Combine all features with vital summary and labels to create the simple dataset if needed

**Inputs**:  
- `6_5_coh3_vital_stats` contains all vital related features
- `6_7_coh3_imputedHWESI` contains all demo, imputed HW and ESI
- merge with `6_8_0_cohort4` -- cohort with labels (and labs) to be consistent with the complex data

**Outputs**: 
- `6_8_coh4_simpledata`: in *wide* format

### Importing R libraries

In [1]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)
# library(mice)
# library(VIM) # for missing data plot

# library(data.table)
# library(Matrix)
# library(caret) # import this before glmnet to avoid rlang version problem
# library(glmnet)
# library(bit64)

# library(slam)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




### Get the dataset for simple models:

In [2]:
# read inputs
datadir6 = "../../DataTD/validation"
valdir = "../../OutputTD/6_validation"

vital_stats <- read.csv(file.path(valdir, '6_5_coh3_vital_stats.csv'))
nrow(vital_stats) #44258

cohort_imp <- read.csv(file.path(valdir, '6_7_coh3_imputedHWESI.csv'))
nrow(cohort_imp) #44258

cohort <- read.csv(file.path(valdir, '6_8_0_cohort4.csv')) #41366
nrow(cohort)

[1] 16704

[1] 16704

[1] 16700

In [3]:
sum(!vital_stats$pat_enc_csn_id_coded %in% cohort_imp$pat_enc_csn_id_coded)
length(setdiff(cohort$pat_enc_csn_id_coded, cohort_imp$pat_enc_csn_id_coded))
length(setdiff(cohort$pat_enc_csn_id_coded, vital_stats$pat_enc_csn_id_coded))

head(vital_stats, n=1)
head(cohort_imp, n=1)
head(cohort, n=1)

[1] 0

[1] 0

[1] 0

,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,DBP_count,DBP_first_val,DBP_fldiff,DBP_IQRx,DBP_last_val,DBP_madx,DBP_maxx,DBP_meanx,DBP_medianx,DBP_minx,⋯,SBP_minx,SBP_mmdiff,SBP_sdx,Temp_count,Temp_first_val,Temp_fldiff,Temp_IQRx,Temp_last_val,Temp_madx,Temp_maxx,Temp_meanx,Temp_medianx,Temp_minx,Temp_mmdiff,Temp_sdx
,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,⋯,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,JC1000116,131295313275,57868578,2020-09-29 22:45:00+00:00,0,5,77,1,1,78,1.4826,80,77,77,73,⋯,97,23,9.418068,2,36.8,0,0.2,36.8,0.29652,37,36.8,36.8,36.6,0.4,0.2828427


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,ESI_i,delta_ESI,gender,age,insurance,English,Height_i,delta_H,Weight_i,delta_W,race.Asian,race.Black,race.Native.American,race.Other,race.Pacific.Islander,race.Unknown,race.White
,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,JC1000116,131295313275,57868578,2020-09-29 22:45:00,0,3,0,1,44,1,0,154,0,73.05,0,0,0,0,1,0,0,0


,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max,previous_icu_visit
,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<chr>
1,JC1000116,131295313275,57868578,2020-09-29 22:45:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0,False


In [4]:
vital_stats <- vital_stats %>% #mutate(admit_time = ymd_hms(admit_time), label_max24 = label) %>%
                select(-inpatient_data_id_coded, -admit_time, -label)
cohort_imp <- cohort_imp %>% #mutate(admit_time = ymd_hms(admit_time), label_max24 = label) %>%
                 select(-inpatient_data_id_coded, -admit_time, -label)
# cohort <- cohort %>% mutate(admit_time = ymd_hms(admit_time))

In [5]:
data_simple <- cohort %>% select(anon_id, pat_enc_csn_id_coded, first_label, death_24hr_recent_label) %>%
                left_join(cohort_imp) %>% left_join(vital_stats) # left_join both, otherwise if full_join, it will retain all rows in vitals
dim(data_simple)
nrow(data_simple %>% select(pat_enc_csn_id_coded) %>% distinct())
colnames(data_simple) #41366

Joining, by = c("anon_id", "pat_enc_csn_id_coded")

Joining, by = c("anon_id", "pat_enc_csn_id_coded")



[1] 16700    81

[1] 16700

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "first_label"             "death_24hr_recent_label"
 [5] "ESI_i"                   "delta_ESI"              
 [7] "gender"                  "age"                    
 [9] "insurance"               "English"                
[11] "Height_i"                "delta_H"                
[13] "Weight_i"                "delta_W"                
[15] "race.Asian"              "race.Black"             
[17] "race.Native.American"    "race.Other"             
[19] "race.Pacific.Islander"   "race.Unknown"           
[21] "race.White"              "DBP_count"              
[23] "DBP_first_val"           "DBP_fldiff"             
[25] "DBP_IQRx"                "DBP_last_val"           
[27] "DBP_madx"                "DBP_maxx"               
[29] "DBP_meanx"               "DBP_medianx"            
[31] "DBP_minx"                "DBP_mmdiff"             
[33] "DBP_sdx"                 "Pulse_count"            
[35] "Pulse_first_val"         "Pulse_fldiff"           
[37] "Pulse_IQRx"              "Pulse_last_val"         
[39] "Pulse_madx"              "Pulse_maxx"             
[41] "Pulse_meanx"             "Pulse_medianx"          
[43] "Pulse_minx"              "Pulse_mmdiff"           
[45] "Pulse_sdx"               "RR_count"               
[47] "RR_first_val"            "RR_fldiff"              
[49] "RR_IQRx"                 "RR_last_val"            
[51] "RR_madx"                 "RR_maxx"                
[53] "RR_meanx"                "RR_medianx"             
[55] "RR_minx"                 "RR_mmdiff"              
[57] "RR_sdx"                  "SBP_count"              
[59] "SBP_first_val"           "SBP_fldiff"             
[61] "SBP_IQRx"                "SBP_last_val"           
[63] "SBP_madx"                "SBP_maxx"               
[65] "SBP_meanx"               "SBP_medianx"            
[67] "SBP_minx"                "SBP_mmdiff"             
[69] "SBP_sdx"                 "Temp_count"             
[71] "Temp_first_val"          "Temp_fldiff"            
[73] "Temp_IQRx"               "Temp_last_val"          
[75] "Temp_madx"               "Temp_maxx"              
[77] "Temp_meanx"              "Temp_medianx"           
[79] "Temp_minx"               "Temp_mmdiff"            
[81] "Temp_sdx"

In [6]:
summary(data_simple)

   anon_id          pat_enc_csn_id_coded  first_label    
 Length:16700       Min.   :1.313e+11    Min.   :0.0000  
 Class :character   1st Qu.:1.313e+11    1st Qu.:0.0000  
 Mode  :character   Median :1.313e+11    Median :0.0000  
                    Mean   :1.313e+11    Mean   :0.1064  
                    3rd Qu.:1.313e+11    3rd Qu.:0.0000  
                    Max.   :1.313e+11    Max.   :1.0000  
                                         NA's   :1       
 death_24hr_recent_label     ESI_i         delta_ESI            gender      
 Min.   :0.00000         Min.   :1.000   Min.   :0.000000   Min.   :0.0000  
 1st Qu.:0.00000         1st Qu.:2.000   1st Qu.:0.000000   1st Qu.:0.0000  
 Median :0.00000         Median :3.000   Median :0.000000   Median :0.0000  
 Mean   :0.09677         Mean   :2.558   Mean   :0.005389   Mean   :0.4722  
 3rd Qu.:0.00000         3rd Qu.:3.000   3rd Qu.:0.000000   3rd Qu.:1.0000  
 Max.   :1.00000         Max.   :5.000   Max.   :1.000000   Max.   :1.0000

In [7]:
# write.csv(cohort_final, file.path(datadir, 'cohort_final.csv'), row.names = FALSE)
write.csv(data_simple, file.path(valdir, "6_8_coh4_simpledata.csv"), row.names=FALSE)